In [ ]:
import requests
import urllib.parse as ulp
import mysql.connector

mydb = mysql.connector.connect(
host="web5.ces.census.gov",
user="##",
password="##",
database="pmt"
)
mycursor = mydb.cursor()

sql = "SET collation_connection = 'utf8_general_ci'"
mycursor.execute(sql)
mydb.commit()
sql1 = "ALTER TABLE api_core CONVERT TO CHARACTER SET utf8 COLLATE utf8_general_ci"
mycursor.execute(sql1)
mydb.commit()

In [1]:
# https://doaj.org/api/v2/search/articles/%22Jack%20Johnson%22?page=1&pageSize=10

class DOAJ:
    
    def sendToDB(self, proj_id, results, hits):
        #print(proj_id)
        mydb = mysql.connector.connect(
        host="web5.ces.census.gov",
        user="##",
        password="##",
        database="pmt"
        )

        mycursor = mydb.cursor()
        
        numPrint = 100
        if(hits < 100):
            numPrint = hits
        #print("numPrint = " + str(numPrint))
        for i in range(numPrint):
            title = str(results['results'][i]['bibjson']['title'])
            doi_array = results['results'][i]['bibjson']['identifier']
            res = next((sub for sub in doi_array if sub['type'] == 'doi'), None)
            doi = res['id']
            authors = results['results'][i]['bibjson9author']
            pub_year = str(results['results'][i]['bibjson']['year'])
            pub_month = str(results['results'][i]['bibjson']['month'])
            keywords = str(results['results'][i]['bibjson'].get('keywords'))
            abstract = str(results['results'][i]['bibjson']['abstract'])
            full_text_url = str(results['results'][i]['bibjson']['link'][0]['url'])
            
            sql_dup = "SELECT * FROM api_doaj WHERE title = %s AND abstract = %s AND proj_id = %s"
            data = (title, abstract, proj_id, )
            mycursor.execute(sql_dup, data)
            duplicates = mycursor.fetchall()

            if len(duplicates) == 0:
                sql = "INSERT INTO api_doaj (proj_id, title, doi, pub_year, pub_month, keywords, abstract, full_text_url) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
                val = (proj_id, title, doi, pub_year, pub_month, keywords, abstract, full_text_url)
                
                mycursor.execute(sql, val)
                pub_id = mycursor.lastrowid
                print("pub_id: " +str(pub_id))

                mydb.commit()

                print(mycursor.rowcount, "record inserted.")

                for j in range(len(author)):
                    sql_part = "INSERT INTO api_doaj_participants (pub_id, proj_id, fullname) VALUES (%s, %s, %s)"
                    val_part = (pub_id, proj_id, author[j])

                    mycursor.execute(sql_part, val_part)

                    mydb.commit()

                    print(mycursor.rowcount, "participant record inserted.")
                    
            else:
                print("duplicate value, not added")
                
        sql_update_doi = "UPDATE api_doaj SET doi=NULL WHERE doi='None'"
        mycursor.execute(sql_update_doi)
        mydb.commit()
        sql_update_title = "UPDATE api_doaj SET title=NULL WHERE title='None'"
        mycursor.execute(sql_update_title)
        mydb.commit()
        print("changed None values to null in doi, title")
        
class DOAJSearch(DOAJ):
    
    def search(self, proj_id, _path, _query):
        header = ["Accept: application/json"]
        url = "https://doaj.org/api/v2"
        #print(url+_path+ulp.quote(_query)+"?page=1&pageSize=20&"+header[1])
        request = requests.get(url+_path+_query+"?page=1&pageSize=20")
        
        if (request.status_code == 200):
            data = [proj_id, request.json()]
            return data
    
    def searchArticles(self, proj_id, _query):
        return self.search(proj_id, "/search/articles/", _query)
    
    def searchJournals(self, _query):
        return self.search(proj_id, "/search/journals/", _query)
        

In [ ]:
api = DOAJSearch()
results = api.search(123, '"Lauren Fang"')
#print(results)
print("Total hits: " + str(results[1]['totalHits']))
print("proj_id: " +str(results[0]))
hits = results[1]['totalHits']
proj_id = str(results[0])
#print(results)
api.sendToDB(proj_id, results, hits)
#print(results)

In [17]:
api = DOAJ()
results = api.searchArticles('"Jack Johnson"')
#print(results)
print("Total hits: " + str(results['total']))
# title = str(results[1]['results'][i]['bibjson']['title'])
# doi_array = str(results[1]['results'][i]['bibjson']['identifier'])
# res = next((sub for sub in doi_array if sub['type'] == 'doi'), None)
# doi = res['id']
# author = results[1]['results'][i]['bibjson']['author'][j]['name']
# pub_year = str(results[1]['results'][i]['bibjson']['year'])
# pub_month = str(results[1]['results'][i]['bibjson']['month'])
# keywords = str(results[1]['results'][i]['bibjson']['keywords'])
# abstract = str(results[1]['results'][i]['bibjson']['abstract'])
# full_text_url = str(results[1]['results'][i]['bibjson']['link']['url'])
i=0
title = str(results['results'][i]['bibjson']['title'])
doi_array = results['results'][i]['bibjson']['identifier']
res = next((sub for sub in doi_array if sub['type'] == 'doi'), None)
doi = res['id']
authors = results['results'][i]['bibjson9author']
pub_year = str(results['results'][i]['bibjson']['year'])
pub_month = str(results['results'][i]['bibjson']['month'])
keywords = str(results['results'][i]['bibjson'].get('keywords'))
abstract = str(results['results'][i]['bibjson']['abstract'])
full_text_url = str(results['results'][i]['bibjson']['link'][0]['url'])
print("title: "+title)
print("doi: "+doi)
print("authors: ")
for author in authors:
    print(author['name'])
print("pub_year: "+pub_year)
print("pub_month: "+pub_month)
print("keywords: "+keywords)
print("abstract: "+abstract)
print("full_text_url: "+full_text_url)
# for result in results:
#     print(result['authors'])

Total hits: 5
title: Distributed Regression Analysis Application in Large Distributed Data Networks: Analysis of Precision and Operational Performance
doi: 10.2196/15073
authors: 
Her, Qoua
Malenfant, Jessica
Zhang, Zilu
Vilk, Yury
Young, Jessica
Tabano, David
Hamilton, Jack
Johnson, Ron
Raebel, Marsha
Boudreau, Denise
Toh, Sengwee
pub_year: 2020
pub_month: 6
keywords: None
abstract: 
          
            BackgroundA distributed data network approach combined with distributed regression analysis (DRA) can reduce the risk of disclosing sensitive individual and institutional information in multicenter studies. However, software that facilitates large-scale and efficient implementation of DRA is limited.
            ObjectiveThis study aimed to assess the precision and operational performance of a DRA application comprising a SAS-based DRA package and a file transfer workflow developed within the open-source distributed networking software PopMedNet in a horizontally partitioned distrib